# Notebook to import Sonata-based files into the database

In [ ]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from tqdm.notebook import tqdm
import numpy as np

db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_schema = 'vasculature'

db_connection_string = 'postgresql+psycopg2://%s:%s@%s:5432/%s' % (db_user, db_password, db_host, db_name)
print('Connection string: ' + db_connection_string + ', schema: ' + db_schema)

engine = create_engine(db_connection_string)
conn = engine.connect()

In [ ]:
import libsonata
data_folder = os.getenv('VASCULATURE_DATA_FOLDER')
print(data_folder)

## Nodes

In [ ]:
nodes = libsonata.NodeStorage(os.path.join(data_folder, 'report_vasculature_6_entry_nodes_new.h5'))
vasculature_population = nodes.open_population('vasculature')
vasculature_population.attribute_names
ids = np.arange(0, vasculature_population.size)
start_xs = vasculature_population.get_attribute('start_x', list(ids))
start_ys = vasculature_population.get_attribute('start_y', list(ids))
start_zs = vasculature_population.get_attribute('start_z', list(ids))
start_ds = vasculature_population.get_attribute('start_diameter', list(ids))
section_ids = vasculature_population.get_attribute('section_id', list(ids))
graph_ids = vasculature_population.get_attribute('subgraph_id', list(ids))
entry_node_ids = vasculature_population.get_attribute('entry_edges', list(ids))
pair_ids = vasculature_population.get_attribute('pairs', list(ids))
# bifurcations = vasculature_population.get_attribute('edges_bifurcations', list(ids))

In [ ]:
# Population
population_description = 'Vasculature'
population_guid = 1

In [ ]:
sql_command = "INSERT INTO " + db_schema +  ".population VALUES (%d, '%s');" % (population_guid, population_description)
conn.execute(sql_command, conn)

In [ ]:
# Nodes
print('Deleting existing nodes')
with Session(engine) as session:
    session.execute(
        'DELETE FROM %s.node WHERE population_guid=:population_guid' % db_schema,
        {
            'population_guid': population_guid
        }
    )
    session.commit()

print('Populating nodes')
duplicated_nodes = list()
with Session(engine) as session:
    for i in tqdm(range(vasculature_population.size)):
        session.execute(
            'INSERT INTO %s.node VALUES (:guid, :population_guid, :x, :y, :z, :radius, :section_guid, :sub_graph_guid, :pair_guid, :entry_node_guid)' % db_schema,
            {
                'guid': i,
                'population_guid': population_guid,
                'x': start_xs[i],
                'y': start_ys[i],
                'z': start_zs[i],
                'radius': start_ds[i] * 0.5,
                'section_guid': int(section_ids[i]),
                'sub_graph_guid': int(graph_ids[i]),
                'pair_guid': int(pair_ids[i]),
                'entry_node_guid': int(entry_node_ids[i]),
            }
        )
        if i % 1000 == 999:
            session.commit()
    session.commit()

## Reports

In [ ]:
def import_simulation_report(simulation_report_guid, population_guid, description, path):
    print('Importing %s from %s' % (description, path))

    reader = libsonata.ElementReportReader(path)
    reader.get_population_names()
    vasculature_report_population = reader['vasculature']

    # Report
    with Session(engine) as session:
        session.execute(
            'DELETE FROM %s.simulation_report WHERE population_guid=:population_guid AND simulation_report_guid=:simulation_report_guid' % db_schema,
            {
                'population_guid': population_guid,
                'simulation_report_guid': simulation_report_guid
            }
        )
        session.commit()

    with Session(engine) as session:
        session.execute(
            'INSERT INTO %s.simulation_report VALUES (:population_guid, :simulation_report_guid, :description, :start_time, :end_time, :time_step, :time_units, :data_units)' % db_schema,
            {
                'population_guid': population_guid,
                'simulation_report_guid': simulation_report_guid,
                'description' : description,
                'start_time': vasculature_report_population.times[0],
                'end_time': vasculature_report_population.times[1],
                'time_step': vasculature_report_population.times[2],
                'time_units': vasculature_report_population.time_units,
                'data_units': vasculature_report_population.data_units
            }
        )
        session.commit()    

    # Report simulation values
    with Session(engine) as session:
        session.execute(
            'DELETE FROM %s.simulation_time_series WHERE population_guid=:population_guid AND simulation_report_guid=:simulation_report_guid' % db_schema,
            {
                'population_guid': population_guid,
                'simulation_report_guid': simulation_report_guid
            }
        )
        session.commit()

    data = vasculature_report_population.get()
    for i in range(len(data.times)):
        frame_data = data.data[i]
    
        buffer = np.array(frame_data, dtype=np.float32).tobytes()
        with Session(engine) as session:
            session.execute(
                'INSERT INTO %s.simulation_time_series VALUES (:population_guid, :simulation_report_guid, :frame, :values)' % db_schema,
                {
                    'population_guid': population_guid,
                    'simulation_report_guid': simulation_report_guid,
                    'frame': i,
                    'values': buffer
                }
            )
            session.commit()    

In [ ]:
import_simulation_report(
    0, population_guid, 'Flows', os.path.join(data_folder, 'v2', 'report_flows.h5'))
import_simulation_report(
    1, population_guid, 'Pressures', os.path.join(data_folder, 'v2', 'report_pressures.h5'))
import_simulation_report(
    2, population_guid, 'Radii', os.path.join(data_folder, 'v2', 'report_radii.h5'))    

## Connectome

In [ ]:
data_folder = '/gpfs/bbp.cscs.ch/project/proj62/scratch/ngv_circuits/20210325/build/sonata/edges'
connectome_db_schema = 'connectome'

glio_vascular_edges = libsonata.EdgeStorage(os.path.join(data_folder, 'gliovascular.h5'))
glio_vascular_population = glio_vascular_edges.open_population('gliovascular')
guids = list(np.arange(0, glio_vascular_population.size))
endfoot_guids = glio_vascular_population.get_attribute('endfoot_id', guids)
astrocyte_guids = glio_vascular_population.target_nodes(guids)
astrocyte_section_guids = glio_vascular_population.get_attribute('astrocyte_section_id', guids)
vasculature_node_guids = glio_vascular_population.source_nodes(guids)
vasculature_section_guids = glio_vascular_population.get_attribute('vasculature_section_id', guids)
vasculature_segment_guids = glio_vascular_population.get_attribute('vasculature_segment_id', guids)
endfoot_compartment_lengths = glio_vascular_population.get_attribute('endfoot_compartment_length', guids)
endfoot_compartment_diameters = glio_vascular_population.get_attribute('endfoot_compartment_diameter', guids)


In [ ]:
glio_vascular_population.attribute_names

In [ ]:
endfoot_compartment_lengths

In [ ]:
populations = [
    [0, 'astrocytes', 'NGV Astrocytes'],
    [1, 'vasculature', 'NGV vasculature']
]
with Session(engine) as session:
    for i in tqdm(range(len(populations))):
        session.execute(
            'INSERT INTO %s.population VALUES(:guid, :schema, :description)' % connectome_db_schema,
            {
                'guid': populations[i][0],
                'schema': populations[i][1],
                'description': populations[i][2],
            }
        )
        session.commit()


In [ ]:
print('Deleting existing glio vascular entries')
with Session(engine) as session:
    session.execute(
        'DELETE FROM %s.glio_vascular WHERE astrocyte_population_guid=:astrocyte_population_guid AND vasculature_population_guid=:vasculature_population_guid' % connectome_db_schema,
        {
            'astrocyte_population_guid': populations[0][0],
            'vasculature_population_guid': populations[1][0],
        }
    )
    session.commit()

print('Populating glio vascular entries')
duplicated_nodes = list()
with Session(engine) as session:
    for i in tqdm(range(len(guids))):
        try:
            session.execute(
                'INSERT INTO %s.glio_vascular VALUES (:guid, :astrocyte_population_guid, :vasculature_population_guid, :astrocyte_guid, :astrocyte_section_guid, :vasculature_node_guid, :vasculature_section_guid, :vasculature_segment_guid, :endfoot_compartment_length, :endfoot_compartment_diameter)' % connectome_db_schema,
                {
                    'guid': int(endfoot_guids[i]),
                    'astrocyte_population_guid': populations[0][0],
                    'vasculature_population_guid': populations[1][0],
                    'astrocyte_guid': int(astrocyte_guids[i]),
                    'astrocyte_section_guid': int(astrocyte_section_guids[i]),
                    'vasculature_node_guid': int(vasculature_node_guids[i]),
                    'vasculature_section_guid': int(vasculature_section_guids[i]),
                    'vasculature_segment_guid': int(vasculature_segment_guids[i]),
                    'endfoot_compartment_length': float(endfoot_compartment_lengths[i]),
                    'endfoot_compartment_diameter': float(endfoot_compartment_diameters[i])
                }
            )
            session.commit()
        except Exception as e:
            session.commit()
            print(e)
    session.commit()
